In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Data exploration and preprocessnig

In [13]:
data_frame = pd.read_csv("data/train.csv")

In [14]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


Define two functions for later use.


In [15]:
print("from: long {} lat {}".format((data_frame.dropoff_latitude.min()), (data_frame.dropoff_longitude.min()))  )
print("to: long {} lat {}".format(max(data_frame.dropoff_latitude), max(data_frame.dropoff_longitude))  )

from: long 32.1811408996582 lat -121.9333038330078
to: long 43.92102813720703 lat -61.33552932739258


In [16]:
print("mean: long {} lat {}".format((data_frame.dropoff_latitude.mean()), (data_frame.dropoff_longitude.mean()))  )

mean: long 40.7517995149002 lat -73.9734159469458


In [17]:
def euklidian_dist(a_x,a_y,b_x,b_y):
    return np.sqrt((a_x - b_x)**2 + (a_y - b_y)**2)

def manhattan_dist(a_x,a_y,b_x,b_y):
    return np.absolute(a_x - b_x) + np.absolute(a_y - b_y)

Here we define our custom fields:
    1. Transform field pickup_datetime to extract information about: month, day of the month, day of the week 
        and hour.
        
    2. Euklidian distance between pickup place and dropoff place.
    
    3. Mahattan distance between pickup place and dropoff place, because we operate in NYC.
    
    4. We also transform trip_duration field with log function.

In [18]:
data_frame['datetime'] = pd.to_datetime(data_frame.pickup_datetime)
data_frame['day_of_week'] = data_frame.datetime.dt.dayofweek
data_frame['hour'] = data_frame.datetime.dt.hour
data_frame['day_of_month'] = data_frame.datetime.dt.day
data_frame['month'] = data_frame.datetime.dt.month
data_frame['euklidian_distance'] = euklidian_dist(data_frame.dropoff_latitude,  data_frame.dropoff_longitude, data_frame.pickup_latitude,data_frame.pickup_longitude)
data_frame['manhattan_distance'] = manhattan_dist(data_frame.dropoff_latitude,  data_frame.dropoff_longitude, data_frame.pickup_latitude,data_frame.pickup_longitude)
data_frame['log_trip_duration'] = np.log(data_frame.trip_duration)

In [19]:
#actual data frame
data_frame[:11]

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,datetime,day_of_week,hour,day_of_month,month,euklidian_distance,manhattan_distance,log_trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2016-03-14 17:24:55,0,17,14,3,0.017680,0.019859,6.120297
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2016-06-12 00:43:35,6,0,12,6,0.020456,0.026478,6.496775
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,2016-01-19 11:35:24,1,11,19,1,0.059934,0.080158,7.661056
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,2016-04-06 19:32:31,2,19,6,4,0.013438,0.015480,6.061457
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,2016-03-26 13:30:55,5,13,26,3,0.010690,0.010818,6.075346
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443,2016-01-30 22:01:40,5,22,30,1,0.011572,0.016212,6.093570
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341,2016-06-17 22:34:59,4,22,17,6,0.014133,0.019669,5.831882
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551,2016-05-21 07:54:58,5,7,21,5,0.059801,0.084026,7.346655
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255,2016-05-27 23:12:23,4,23,27,5,0.014790,0.019279,5.541264
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225,2016-03-10 21:45:01,3,21,10,3,0.046355,0.053699,7.110696


We tranform use log function on trip_duration, because it is supposed reduce skeewnes.

More on that here
https://becominghuman.ai/how-to-deal-with-skewed-dataset-in-machine-learning-afd2928011cc

All I know is that can help with regression in some cases, and it was true in ours.

Here is what is correlation between given fields and a trip_duration, it gives us information how each field influences the output:


And here is correlation betwen given fields and log_trip_duration:


We pick fields for later training. We have decided to use both euklidian and manhattan distance, because this configuration gave us the best results.

In [34]:
data_X   = data_frame[[
        'pickup_longitude', 'pickup_latitude',
        'dropoff_longitude', 'dropoff_latitude',
        'month', 'hour',
        'euklidian_distance',
        'manhattan_distance',
    
       ]].values



data_y = data_frame['log_trip_duration'].values

In [35]:

train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.2)

In [36]:
train_X

array([[ -7.39649582e+01,   4.07725754e+01,  -7.39607239e+01, ...,
          1.80000000e+01,   7.31295375e-03,   1.01966858e-02],
       [ -7.39895401e+01,   4.07400093e+01,  -7.39935684e+01, ...,
          9.00000000e+00,   1.26409979e-02,   1.60102844e-02],
       [ -7.37900467e+01,   4.06469383e+01,  -7.39790802e+01, ...,
          1.50000000e+01,   2.21157954e-01,   3.03825378e-01],
       ..., 
       [ -7.39905090e+01,   4.07410583e+01,  -7.39750214e+01, ...,
          2.00000000e+01,   1.68727061e-02,   2.21824646e-02],
       [ -7.38638306e+01,   4.07697868e+01,  -7.40070496e+01, ...,
          1.70000000e+01,   1.46451808e-01,   1.73820496e-01],
       [ -7.39623795e+01,   4.07706490e+01,  -7.39758301e+01, ...,
          1.10000000e+01,   1.51577945e-02,   2.04391479e-02]])

We scale our input. By calculating mean of each column, and then dividing each element by mean of column it is in. After that operation all input fields should be moreover the same.

In [37]:
means = np.array([np.mean(train_X[:,i]) for i in range(len(train_X[0]))])
train_X = train_X / means
test_X = test_X / means
print(means)


[ -7.39734503e+01   4.07509192e+01  -7.39733898e+01   4.07517839e+01
   3.51708479e+00   1.36067434e+01   3.55015215e-02   4.59241450e-02]


# Production Model

**4 .Gradient Boosted Trees**

In [39]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error

In [14]:
n = 50000
model1 = GradientBoostingRegressor(criterion='mse')
%timeit model1.fit(train_X[:n],train_y[:n])
joblib.dump(model1,"models/production50k.pkl")
print("DONE")

4.88 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
DONE


In [15]:
n = 100000
model2 = GradientBoostingRegressor(criterion='mse')
%timeit model2.fit(train_X[:n],train_y[:n])
joblib.dump(model2,"models/production100k.pkl")
print("DONE")

13.5 s ± 663 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
DONE


In [16]:
n = 200000
model3 = GradientBoostingRegressor(criterion='mse')
%timeit model3.fit(train_X[:n],train_y[:n])
joblib.dump(model3,"models/production200k.pkl")
print("DONE")

29.9 s ± 383 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
DONE


In [17]:
n = 400000
model4 = GradientBoostingRegressor(criterion='mse')
%timeit model4.fit(train_X[:n],train_y[:n])
joblib.dump(model4,"models/production400k.pkl")
print("DONE")

1min 12s ± 1.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
DONE


In [27]:
model4 = GradientBoostingRegressor(criterion='mse')
%timeit model4.fit(train_X,train_y)
joblib.dump(model4,"models/production_full.pkl")
print("DONE")

3min 5s ± 9.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
DONE


In [40]:
model4 = joblib.load("models/production_full.pkl")
result = model4.predict(test_X)
print(mean_squared_error(test_y, result))

0.269155333597


# SUMMARY

**The best result we've achieved was lmse 0.26. This is the model we are going to use in Unter service.**

danpisq@Unter